In [1]:
#!/usr/bin/python3
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    MeshcatVisualizer,
    SceneGraph,
    Simulator,
    StartMeshcat,
    Rgba
)
from pydrake.examples import (
    QuadrotorGeometry,
    QuadrotorPlant,
)

from quad_traj_opt.trajectory_optimization import TrajectoryOptimization
from quad_traj_opt.differential_flatness import trajectory_to_state

import quadrotor_params as p
import waypoint_library


In [2]:
# waypoints = waypoint_library.waypoints_straight
waypoints = waypoint_library.waypoints_flip/3
headings = np.zeros((1, waypoints.shape[1]))

traj_gen = TrajectoryOptimization(waypoints, headings, ts=0.5)
traj_gen.solve()

# t = np.linspace(0, traj_gen.tf, 1000)
dt = .01
t = np.arange(0.0, traj_gen.tf, dt)

In [3]:
running_as_notebook = True
meshcat = StartMeshcat()
builder = DiagramBuilder()
waypoint_vis = traj_gen.get_keyframe_point_cloud( 
    pts_per_edge=p.keyframes_pts_per_edge, half_width=p.keyframes_half_width)
traj_vis = traj_gen.get_trajectory_point_cloud(t)

plant = builder.AddSystem(QuadrotorPlant(
    m_arg=p.m_kg,
    L_arg=p.L_m,
    I_arg=p.J,
    kF_arg=p.kF,
    kM_arg=p.kM
))

meshcat.SetObject("/point_cloud/", traj_vis, point_size=.05, rgba=Rgba(1.0, 0.0, 0.5, 1.0))
meshcat.SetObject("/waypoints/", waypoint_vis, point_size=.05, rgba=Rgba(0.0, 0.5, 0.5, 1.0))

# Set up visualization in MeshCat
scene_graph = builder.AddSystem(SceneGraph())
QuadrotorGeometry.AddToBuilder(
    builder, plant.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# end setup for visualization

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
context = simulator.get_mutable_context()

INFO:drake:Meshcat listening for connections at http://localhost:7008


In [4]:
# Simulate
for i in range(3):
    context.SetTime(0.0)
    simulator.Initialize()
    context.SetContinuousState(np.zeros(12)) 
    
    for ti in t[0:]:
        try:
            simulator.AdvanceTo(ti)
        except:
            simulator.Initialize()
        traj_matrix = traj_gen.eval_full_trajectory(ti)
        x = trajectory_to_state(traj_matrix, plant)
        context.SetContinuousState(x)